In [110]:
# pip install sentence-transformers

In [111]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

In [112]:
file_path = 'fmp-dump.csv'
data = pd.read_csv(file_path)

In [113]:
# Normalizing 'Faculty' table
faculty_df = data[['First Name', 'Last Name', 'Job Title']].drop_duplicates().reset_index(drop=True)
faculty_df['FacultyID'] = faculty_df.index + 1  # Adding primary key for each faculty

In [114]:
# Normalizing 'Department' table
department_df = data[['Department', 'School']].drop_duplicates().reset_index(drop=True)
department_df['DepartmentID'] = department_df.index + 1  # Adding primary key for each department

# Maping each faculty to their department
faculty_df = faculty_df.merge(data[['First Name', 'Last Name', 'Department']],
                              on=['First Name', 'Last Name'], how='left')
faculty_df = faculty_df.merge(department_df[['Department', 'DepartmentID']],
                              on='Department', how='left').drop(columns=['Department'])

In [115]:
# Normalizing 'Expertise' table
expertise_df = data['Area of Expertise'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
expertise_df = expertise_df.reset_index().rename(columns={0: 'Expertise'})
expertise_df = expertise_df.drop_duplicates().reset_index(drop=True)
expertise_df['ExpertiseID'] = expertise_df.index + 1  # Adding primary key for expertise

In [116]:
# Normalizing 'Country' table
country_df = data['Countries of Interest'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
country_df = country_df.reset_index().rename(columns={0: 'Country'})
country_df = country_df.drop_duplicates().reset_index(drop=True)
country_df['CountryID'] = country_df.index + 1  # Adding primary key for countries

In [117]:
# Maping each faculty to their expertise and countries of interest through relational tables
faculty_expertise_df = data[['First Name', 'Last Name', 'Area of Expertise']].merge(faculty_df[['First Name', 'Last Name', 'FacultyID']],
                                                                                     on=['First Name', 'Last Name'])
faculty_expertise_df = faculty_expertise_df[['FacultyID', 'Area of Expertise']].drop_duplicates()

# Exploding expertise to match 3NF structure
faculty_expertise_df = faculty_expertise_df.assign(Expertise=faculty_expertise_df['Area of Expertise'].str.split(', ')).explode('Expertise')
faculty_expertise_df = faculty_expertise_df.merge(expertise_df[['Expertise', 'ExpertiseID']], on='Expertise').drop(columns=['Area of Expertise', 'Expertise'])

# Mapping faculty to countries of interest
faculty_country_df = data[['First Name', 'Last Name', 'Countries of Interest']].merge(faculty_df[['First Name', 'Last Name', 'FacultyID']],
                                                                                     on=['First Name', 'Last Name'])
faculty_country_df = faculty_country_df[['FacultyID', 'Countries of Interest']].drop_duplicates()

# Exploding countries of interest for 3NF
faculty_country_df = faculty_country_df.assign(Country=faculty_country_df['Countries of Interest'].str.split(', ')).explode('Country')
faculty_country_df = faculty_country_df.merge(country_df[['Country', 'CountryID']], on='Country').drop(columns=['Countries of Interest', 'Country'])

In [118]:
# Expertise Clustering Using Transformer and kmeans clustering to boil down expertise count from 6604 to ~40

# Extracting unique expertise fields
expertise_fields = expertise_df['Expertise'].tolist()

# Step 1: Generating word embeddings using Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
X = model.encode(expertise_fields)

# Step 2: Determining the optimal number of clusters using silhouette score
silhouette_scores = []
k_values = range(10, 50, 10)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))

# Selecting the optimal number of clusters for kmeans clustering
optimal_k = k_values[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k}")

# Step 3: Fitting KMeans with optimal cluster count
kmeans = KMeans(n_clusters=optimal_k, random_state=0)
kmeans.fit(X)
labels = kmeans.labels_

# Step 4: Creating a DataFrame with expertise areas and their assigned cluster
expertise_df['Category'] = labels

# Step 5: Generating a single representative name for each cluster
def get_top_term_per_cluster(expertise_fields, X, labels):
    """
    Get top term for each cluster based on word vector centroids.
    """
    cluster_names = {}
    for cluster_num in set(labels):
        # Selecting rows in the current cluster
        cluster_indices = [i for i, label in enumerate(labels) if label == cluster_num]
        cluster_expertise = [expertise_fields[i] for i in cluster_indices]

        # Using the most frequent term in the cluster as the representative name
        top_term = max(set(cluster_expertise), key=cluster_expertise.count)
        cluster_names[cluster_num] = top_term

    return cluster_names

# Getting single term cluster names
cluster_names = get_top_term_per_cluster(expertise_fields, X, labels)

# Adding cluster names to expertise_df
expertise_df['Category_Name'] = expertise_df['Category'].map(cluster_names)

# Merging back with faculty_expertise_df to include the expertise category
faculty_expertise_df = faculty_expertise_df.merge(expertise_df[['ExpertiseID', 'Category_Name']], on='ExpertiseID', how='left')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Optimal number of clusters: 40


In [119]:
expertise_df.drop(columns=['index'], inplace=True)
country_df.drop(columns=['index'], inplace=True)

In [120]:
print("Faculty Table:\n", faculty_df)
print("\nDepartment Table:\n", department_df)
print("\nExpertise Table:\n", expertise_df)
print("\nCountry Table:\n", country_df)
print("\nFaculty Expertise Table with Category:\n", faculty_expertise_df)
print("\nFaculty Country Table:\n", faculty_country_df)

Faculty Table:
      First Name Last Name                     Job Title  FacultyID  \
0     Elizabeth    Catlos           Associate Professor          1   
1     Elizabeth    Catlos           Associate Professor          1   
2      Benjamin     Gregg                     Professor          2   
3      Benjamin     Gregg                     Professor          2   
4           Amy       Liu                     Professor          3   
...         ...       ...                           ...        ...   
1593  Katherine     Freer           Assistant Professor       1290   
1594     Jessie   Contour           Associate Professor       1291   
1595       Gray    Garmon           Assistant Professor       1292   
1596      Farya  Phillips  Research Assistant Professor       1293   
1597     Jiabao        Li           Assistant Professor       1294   

      DepartmentID  
0                1  
1               61  
2                2  
3              121  
4                2  
...            ..

In [121]:
faculty_df.to_csv("faculty.csv", index=False)
department_df.to_csv("department.csv", index=False)
expertise_df.to_csv("expertise.csv", index=False)
country_df.to_csv("countries.csv", index=False)
faculty_expertise_df.to_csv("faculty_expertise.csv", index=False)
faculty_country_df.to_csv("faculty_country_of_interest.csv", index=False)